In [5]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load the labeled dataset
with open('training_articles.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Parse data: use 'text' as input and 'relevant' as label (convert to int if needed)
df = pd.DataFrame([{'text': item.get('text', ''), 'label': 1 if str(item.get('relevant', '')).lower() == 'true' else 0} for item in data])

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train a logistic regression classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)

# Evaluate the classifier
y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80        26
           1       1.00      0.07      0.13        14

    accuracy                           0.68        40
   macro avg       0.83      0.54      0.47        40
weighted avg       0.78      0.68      0.57        40



In [2]:
import json
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from sklearn.preprocessing import LabelEncoder

# 1. Load & prepare data once
with open("training_articles.json", "r", encoding="utf-8") as f:
    data = json.load(f)
df = pd.DataFrame([
    {
        "text": item.get("text", ""),
        "label": 1 if str(item.get("relevant", "")).lower() == "true" else 0,
    }
    for item in data
])

# 2. Encode labels
le = LabelEncoder()
df["label_enc"] = le.fit_transform(df["label"])

# 3. Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize_texts(texts, max_length=256):
    return tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    )

# 4. Metrics fn
def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1":       f1_score(labels, preds, average="binary"),
    }

# 5. Prepare cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(df["text"], df["label_enc"]), 1):
    print(f"\n### Fold {fold}")

    # Split into texts & labels
    train_texts = df.loc[train_idx, "text"]
    train_labels = df.loc[train_idx, "label_enc"]
    val_texts   = df.loc[val_idx,   "text"]
    val_labels   = df.loc[val_idx,   "label_enc"]

    # Tokenize
    train_enc = tokenize_texts(train_texts)
    val_enc   = tokenize_texts(val_texts)

    # Dataset wrapper
    class NewsDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels    = torch.tensor(labels.values if hasattr(labels, "values") else labels)
        def __getitem__(self, idx):
            item = {k: v[idx] for k, v in self.encodings.items()}
            item["labels"] = self.labels[idx]
            return item
        def __len__(self):
            return len(self.labels)

    train_ds = NewsDataset(train_enc, train_labels)
    val_ds   = NewsDataset(val_enc,   val_labels)

    # Load fresh model for each fold
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

    # Training args (you can adjust per your GPU/memory)
    args = TrainingArguments(
        output_dir=f"./results/fold{fold}",
        num_train_epochs=2,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        do_train=True,
        do_eval=True,
        logging_steps=10,
        save_steps=0,
        disable_tqdm=False,
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    # Train & eval
    trainer.train()
    metrics = trainer.evaluate()

    # Record F1
    fold_f1 = metrics["eval_f1"]
    fold_results.append(fold_f1)
    print(f"Fold {fold} F1 = {fold_f1:.4f}")

# 6. Summarize
avg_f1 = sum(fold_results) / len(fold_results)
print(f"\nCross-validated F1 scores: {fold_results}")
print(f"Mean F1: {avg_f1:.4f}")


/mnt/raid1/MSCs/AY2425/skaul/miniconda3/envs/scraper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



### Fold 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1057128/199679458.py:98: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,0.644800
20,0.527300
30,0.311800
40,0.292400


Fold 1 F1 = 0.7500

### Fold 2


...

Step,Training Loss
10,0.631300
20,0.458400
30,0.198400
40,0.314400


Fold 4 F1 = 0.9091

### Fold 5


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1057128/199679458.py:98: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,0.645800
20,0.556100
30,0.429600
40,0.330100


Fold 5 F1 = 0.7619

Cross-validated F1 scores: [0.75, 0.8888888888888888, 0.56, 0.9090909090909091, 0.7619047619047619]
Mean F1: 0.7740


In [5]:
# Tokenize entire corpus
full_enc = tokenize_texts(df["text"])
full_ds  = NewsDataset(full_enc, df["label_enc"])

# Load a fresh model
final_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2
)

# (Optional) freeze layers if you found that helpful
for i, layer in enumerate(final_model.bert.encoder.layer):
    if i < 8:
        for p in layer.parameters():
            p.requires_grad = False

# TrainingArguments for the “final” run
final_args = TrainingArguments(
    output_dir="./final_model",
    num_train_epochs=3,                # train a bit longer now
    per_device_train_batch_size=8,
    learning_rate=3e-5,                # or whatever your best CV LR was
    weight_decay=0.01,                 # from your inner CV
    logging_steps=50,
    save_steps=500,
)

# Trainer & train
final_trainer = Trainer(
    model=final_model,
    args=final_args,
    train_dataset=full_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,   # so you can still call evaluate()
)
final_trainer.train()

# Save model & tokenizer
final_trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1057128/2215723649.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  final_trainer = Trainer(


Step,Training Loss
50,0.444800


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/vocab.txt',
 './final_model/added_tokens.json')

In [6]:
import torch
import torch.nn.functional as F

# Load the final model back (just to demo inference)
from transformers import BertForSequenceClassification
inference_model = BertForSequenceClassification.from_pretrained("./final_model")
inference_model.eval()

# Pick 5 random articles
sample = df.sample(5, random_state=123).reset_index(drop=True)
texts  = sample["text"].tolist()

# Tokenize & to device
enc = tokenizer(texts, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
with torch.no_grad():
    logits = inference_model(enc["input_ids"], attention_mask=enc["attention_mask"]).logits
probs = F.softmax(logits, dim=-1)

# Map back to label names
inv_map = {i: cls for i, cls in enumerate(le.classes_)}

for i, txt in enumerate(texts):
    pred_id   = torch.argmax(probs[i]).item()
    confidence= probs[i, pred_id].item()
    print(f"--- Example {i+1} ---")
    print(txt[:200].replace("\n"," "), "…")
    print(f"Predicted = {inv_map[pred_id]}  (conf={confidence:.2f})\n")


--- Example 1 ---
Born a few weeks into the Gaza war, infant twins Wesam and Naeem Abu Anza were buried on Sunday, the youngest of 14 members of the same family whom Gaza health authorities say were killed in an Israel …
Predicted = 1  (conf=0.89)

--- Example 2 ---
Volodymyr Zelensky  has hailed the UK’s £2.5 billion military aid package for  Ukraine , as  Rishi Sunak  promised to continue to stand with the country in its fight against Russia. The  Prime Ministe …
Predicted = 0  (conf=0.94)

--- Example 3 ---
Dr. Siegel: I've changed my view of Fetterman and now see his 'personal courage'   Fox News medical contributor Dr. Marc Siegel applauds Democratic Sen. John Fetterman's recent candid interview on his …
Predicted = 0  (conf=0.97)

--- Example 4 ---
Dozens of global traditions could be part of UNESCO's intangible global heritage list this week. These include Italian opera singing and Bangladeshi rickshaw art as well as the Peruvian delicacy of ce …
Predicted = 0  (conf=0.95)

--- 

In [ ]:
import json
import json5
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification

# 1) Paste in your JSON:
with open("articles.json", "r", encoding="utf-8") as f:
    articles = json.load(f)   # <-- json.load on a file, not json.loads on a string

texts    = [a["text"] for a in articles]
titles   = [a["title"] for a in articles]

# 3) Load your fine-tuned model & tokenizer
MODEL_PATH = "./final_model"   # adjust if different
tokenizer  = BertTokenizer.from_pretrained(MODEL_PATH)
model      = BertForSequenceClassification.from_pretrained(MODEL_PATH)
model.eval()

# 4) Tokenize inputs
enc = tokenizer(
    texts,
    padding="max_length",
    truncation=True,
    max_length=256,
    return_tensors="pt",
)

# 5) Run inference
with torch.no_grad():
    logits = model(
        input_ids=enc["input_ids"],
        attention_mask=enc["attention_mask"],
    ).logits

probs       = F.softmax(logits, dim=-1)
pred_ids    = probs.argmax(dim=-1).tolist()
confidences = probs.max(dim=-1).values.tolist()

# 6) Map class IDs back to names
label_map = {0: "irrelevant", 1: "relevant"}

# 7) Print results
for i, title in enumerate(titles):
    print(f"--- Article {i+1} ---")
    print("Title: ", title)
    print(f"Predicted: {label_map[pred_ids[i]]}  (confidence: {confidences[i]:.2f})\n")


# High RECALL, precision not == 100% but close.

--- Article 1 ---
Title:  ‘Kamala Harris, Joe Biden ignored Hindus’: Donald Trump condemns violence against minorities
Predicted: irrelevant  (confidence: 0.97)

--- Article 2 ---
Title:  Israeli firefighting teams battle bushfires near Jerusalem for second day
Predicted: relevant  (confidence: 0.80)

--- Article 3 ---
Title:  NYC broker fee law — which protects tenants from costs — challenged by real estate agents in 11th-hour push
Predicted: irrelevant  (confidence: 0.95)

--- Article 4 ---
Title:  US proposes 60-day ceasefire for Gaza; hostage-prisoner swap, plan shows
Predicted: relevant  (confidence: 0.87)

--- Article 5 ---
Title:  Mother of Palestinian American boy slain in suburban Chicago hate crime testifies at trial
Predicted: irrelevant  (confidence: 0.83)

--- Article 6 ---
Title:  5 things to know for Feb. 26: Air travel safety, Budget blueprint, Ukraine’s minerals, Immigration, Gaza ceasefire
Predicted: irrelevant  (confidence: 0.97)

--- Article 7 ---
Title:  5 things t

In [ ]:
import json
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from sklearn.preprocessing import LabelEncoder

# 1. Load & prepare data
with open("training_articles.json", "r", encoding="utf-8") as f:
    raw = json.load(f)
df = pd.DataFrame([
    {
        "text": item.get("text", ""),
        "label": 1 if str(item.get("relevant", "")).lower() == "true" else 0,
    }
    for item in raw
])

# 2. Encode labels
le = LabelEncoder()
df["label_enc"] = le.fit_transform(df["label"])

# 3. Tokenizer helper
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize_texts(texts, max_length=256):
    return tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    )

# 4. Dataset wrapper
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels    = torch.tensor(labels.values if hasattr(labels, "values") else labels)
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

# 5. Metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "f1":       f1_score(p.label_ids, preds, average="binary"),
    }

# 6. Hyperparameter grid for inner CV
param_grid = [
    {"learning_rate": 2e-5, "weight_decay": 0.0},
    {"learning_rate": 3e-5, "weight_decay": 0.01},
]

outer_skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
outer_scores = []

for fold, (train_idx, test_idx) in enumerate(
    outer_skf.split(df["text"], df["label_enc"]), start=1
):
    print(f"\n=== Outer Fold {fold} ===")
    X_train, y_train = df["text"].iloc[train_idx], df["label_enc"].iloc[train_idx]
    X_test,  y_test  = df["text"].iloc[test_idx],  df["label_enc"].iloc[test_idx]

    # ---- Inner CV to select hyperparameters ----
    best_inner_f1 = -1.0
    best_params   = param_grid[0]  # fallback
    inner_skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=fold)

    for params in param_grid:
        inner_f1s = []
        for inner_tr, inner_val in inner_skf.split(X_train, y_train):
            # Tokenize
            enc_tr = tokenize_texts(X_train.iloc[inner_tr])
            enc_val= tokenize_texts(X_train.iloc[inner_val])
            ds_tr  = NewsDataset(enc_tr, y_train.iloc[inner_tr])
            ds_val = NewsDataset(enc_val, y_train.iloc[inner_val])

            # Load model & freeze first 8 layers
            model = BertForSequenceClassification.from_pretrained(
                "bert-base-uncased", num_labels=2
            )
            for i, layer in enumerate(model.bert.encoder.layer):
                if i < 8:
                    for p in layer.parameters():
                        p.requires_grad = False

            # Legacy TrainingArguments (no evaluation_strategy)
            args = TrainingArguments(
                output_dir=f"./tmp/inner_fold{fold}",
                num_train_epochs=2,
                per_device_train_batch_size=8,
                per_device_eval_batch_size=8,
                learning_rate=params["learning_rate"],
                weight_decay=params["weight_decay"],
                do_train=True,
                do_eval=True,
                logging_steps=10,
                save_steps=0,
                disable_tqdm=True,
            )
            trainer = Trainer(
                model=model,
                args=args,
                train_dataset=ds_tr,
                eval_dataset=ds_val,
                compute_metrics=compute_metrics,
            )
            trainer.train()
            metrics = trainer.evaluate()
            inner_f1s.append(metrics["eval_f1"])

        avg_f1 = sum(inner_f1s) / len(inner_f1s)
        if avg_f1 > best_inner_f1:
            best_inner_f1, best_params = avg_f1, params

    print(f"Best inner F1 = {best_inner_f1:.4f} with {best_params}")

    # ---- Final training on full train, evaluation on outer test ----
    enc_tr_full = tokenize_texts(X_train)
    enc_te_full = tokenize_texts(X_test)
    ds_tr_full  = NewsDataset(enc_tr_full, y_train)
    ds_te_full  = NewsDataset(enc_te_full,  y_test)

    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased", num_labels=2
    )
    for i, layer in enumerate(model.bert.encoder.layer):
        if i < 8:
            for p in layer.parameters():
                p.requires_grad = False

    args = TrainingArguments(
        output_dir=f"./results/fold{fold}",
        num_train_epochs=2,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        learning_rate=best_params["learning_rate"],
        weight_decay= best_params["weight_decay"],
        do_train=True,
        do_eval=True,
        logging_steps=10,
        save_steps=0,
        disable_tqdm=True,
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_tr_full,
        eval_dataset=ds_te_full,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    final_metrics = trainer.evaluate()
    outer_scores.append(final_metrics["eval_f1"])
    print(f"Outer Fold {fold} F1 = {final_metrics['eval_f1']:.4f}")

# Summarize nested CV results
print("\nNested CV F1 per fold:", outer_scores)
print("Mean F1:", sum(outer_scores) / len(outer_scores))



=== Outer Fold 1 ===


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'loss': 0.6463, 'grad_norm': 8.24787712097168, 'learning_rate': 1.3571428571428574e-05, 'epoch': 0.7142857142857143}
{'loss': 0.5087, 'grad_norm': 3.069215774536133, 'learning_rate': 6.4285714285714295e-06, 'epoch': 1.4285714285714286}
{'train_runtime': 6.0999, 'train_samples_per_second': 34.754, 'train_steps_per_second': 4.59, 'train_loss': 0.5816445010049003, 'epoch': 2.0}
{'eval_loss': 0.5504566431045532, 'eval_accuracy': 0.7037037037037037, 'eval_f1': 0.0, 'eval_runtime': 0.2536, 'eval_samples_per_second': 212.959, 'eval_steps_per_second': 27.606, 'epoch': 2.0}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'loss': 0.64, 'grad_norm': 2.9499199390411377, 'learning_rate': 1.3571428571428574e-05, 'epoch': 0.7142857142857143}
{'loss': 0.5572, 'grad_norm': 9.155176162719727, 'learning_rate': 6.4285714285714295e-06, 'epoch': 1.4285714285714286}
{'train_runtime': 6.5471, 'train_samples_per_second': 32.686, 'train_steps_per_second': 4.277, 'train_loss': 0.5644358992576599, 'epoch': 2.0}
{'eval_loss': 0.5360226631164551, 'eval_accuracy': 0.6981132075471698, 'eval_f1': 0.0, 'eval_runtime': 0.2502, 'eval_samples_per_second': 211.79, 'eval_steps_per_second': 27.972, 'epoch': 2.0}


...

{'loss': 0.6515, 'grad_norm': 3.749399185180664, 'learning_rate': 2.0357142857142858e-05, 'epoch': 0.7142857142857143}
{'loss': 0.4168, 'grad_norm': 4.520461082458496, 'learning_rate': 9.642857142857144e-06, 'epoch': 1.4285714285714286}
{'train_runtime': 7.0921, 'train_samples_per_second': 30.174, 'train_steps_per_second': 3.948, 'train_loss': 0.4892325145857675, 'epoch': 2.0}
{'eval_loss': 0.47362592816352844, 'eval_accuracy': 0.7547169811320755, 'eval_f1': 0.3157894736842105, 'eval_runtime': 0.257, 'eval_samples_per_second': 206.233, 'eval_steps_per_second': 27.238, 'epoch': 2.0}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'loss': 0.602, 'grad_norm': 2.950532913208008, 'learning_rate': 2.0357142857142858e-05, 'epoch': 0.7142857142857143}
{'loss': 0.4514, 'grad_norm': 3.5660126209259033, 'learning_rate': 9.642857142857144e-06, 'epoch': 1.4285714285714286}
{'train_runtime': 7.8665, 'train_samples_per_second': 27.204, 'train_steps_per_second': 3.559, 'train_loss': 0.5116022654942104, 'epoch': 2.0}
{'eval_loss': 0.40703240036964417, 'eval_accuracy': 0.7924528301886793, 'eval_f1': 0.47619047619047616, 'eval_runtime': 0.2605, 'eval_samples_per_second': 203.459, 'eval_steps_per_second': 26.872, 'epoch': 2.0}
Best inner F1 = 0.3342 with {'learning_rate': 3e-05, 'weight_decay': 0.01}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'loss': 0.6879, 'grad_norm': 11.426597595214844, 'learning_rate': 2.3250000000000003e-05, 'epoch': 0.5}
{'loss': 0.4246, 'grad_norm': 7.648776531219482, 'learning_rate': 1.575e-05, 'epoch': 1.0}
{'loss': 0.2924, 'grad_norm': 7.088068008422852, 'learning_rate': 8.25e-06, 'epoch': 1.5}
{'loss': 0.3632, 'grad_norm': 5.967757225036621, 'learning_rate': 7.5e-07, 'epoch': 2.0}
{'train_runtime': 9.9736, 'train_samples_per_second': 32.085, 'train_steps_per_second': 4.011, 'train_loss': 0.44202861189842224, 'epoch': 2.0}
{'eval_loss': 0.452415406703949, 'eval_accuracy': 0.775, 'eval_f1': 0.5714285714285714, 'eval_runtime': 0.1986, 'eval_samples_per_second': 201.391, 'eval_steps_per_second': 25.174, 'epoch': 2.0}
Outer Fold 5 F1 = 0.5714

Nested CV F1 per fold: [0.0, 0.782608695652174, 0.5, 0.6666666666666666, 0.5714285714285714]
Mean F1: 0.5041407867494824
